In [1]:
import pickle
from submodlib import FacilityLocationMutualInformationFunction
from submodlib import GraphCutMutualInformationFunction
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from submodlib import FacilityLocationVariantMutualInformationFunction
from submodlib import ConcaveOverModularFunction
from submodlib import LogDeterminantMutualInformationFunction
from submodlib_cpp import ConcaveOverModular

/home/ganesh/vishak/miniconda3/envs/lora/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [2]:
with open('/home/ganesh/vishak/ashutosh/Attribution/Experiments_Attribution/exp_7/embeddings/llama/dict_embedding_posthoc_nq_open_op_answer_input.pkl', 'rb') as f:
    finetune_data_op_embeddings = pickle.load(f)

In [3]:
finetune_data_op_embeddings.shape

(361000, 768)

In [5]:
# with open('../grounded_passages/_ans_context_ft.json', 'r') as myfile:
#     grounded_data=myfile.read()
# obj = json.loads(grounded_data)

In [6]:
# retrieved_contexts={}
# for item in obj:
#     query = item['question']
#     retrieved_contexts[query] = []
#     for context_index in range(100):
#         retrieved_context = item['ctxs'][context_index]['text']
#         retrieved_contexts[query].append(retrieved_context)

In [7]:
# queries = list(retrieved_contexts.keys())

In [ ]:
# with open('predictions_ais_600_inp_qn_op_answer2.pkl', 'rb') as f:
#     predictions = pickle.load(f)

In [9]:
model=SentenceTransformer("gtr-t5-xxl")
model.half()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 1024, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
)

In [10]:
qa = np.load('../embeddings/llama/embedding_answers.npy')
qq = np.load('../embeddings/llama/embedding_questions.npy')

In [12]:
index_list={}
for i in range(0,len(finetune_data_op_embeddings), 100):
    index = int(i/100)
    print(index) 

    answer = qa[index].reshape((1, 768))
    query = qq[index].reshape((1, 768))
    
    qd = np.concatenate([answer])
    #print(len(qd))

    indices=[]
    features = finetune_data_op_embeddings[i:i+100]
    objFL = GraphCutMutualInformationFunction(n=100, num_queries=len(qd), data=features, 
                                        queryData=qd, 
                                        metric="cosine")
    greedyList = objFL.maximize(budget=3,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
    indices.append([i[0] for i in greedyList])
    index_list[index] = indices
    if index>10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


[||||||||||||||||||||]100% [Iteration 3 of 3]

In [13]:
index_list

{0: [[79, 8, 0]],
 1: [[92, 50, 0]],
 2: [[36, 21, 98]],
 3: [[35, 84, 2]],
 4: [[0, 21, 5]],
 5: [[75, 14, 71]],
 6: [[80, 52, 66]],
 7: [[39, 6, 0]],
 8: [[1, 0, 3]],
 9: [[27, 10, 78]],
 10: [[3, 2, 0]],
 11: [[71, 90, 51]]}

In [ ]:
with open('graphcutmi_answertss_3.pkl', 'wb') as handle:
        pickle.dump(index_list, handle)

In [ ]:
index_list={}
for i in range(0,len(finetune_data_op_embeddings), 100):
    index = int(i/100)
    print(index)

    answer = qa[index].reshape((1, 768))
    query = qq[index].reshape((1, 768))
    
    qd = np.concatenate([answer])
    #print(len(qd))

    indices=[]
    features = finetune_data_op_embeddings[i:i+100]
    objFL = FacilityLocationMutualInformationFunction(n=100, num_queries=len(qd), data=features, 
                                                    queryData=qd, metric="cosine", 
                                                    magnificationEta=1.0)
    greedyList = objFL.maximize(budget=3,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
    indices.append([i[0] for i in greedyList])
    index_list[index] = indices

In [ ]:
index_list[0]

In [ ]:
with open('facilitylocmi_answertss_3.pkl', 'wb') as handle:
        pickle.dump(index_list, handle)

In [ ]:
index_list={}
for i in range(0,len(finetune_data_op_embeddings), 100):
    index = int(i/100)
    #print(index)

    answer = qa[index].reshape((1, 768))
    query = qq[index].reshape((1, 768))
    
    qd = np.concatenate([answer])
    #print(len(qd))

    indices=[]
    features = finetune_data_op_embeddings[i:i+100]
    objFL = LogDeterminantMutualInformationFunction(n=100, num_queries=len(qd), data=features, 
                                                    queryData=qd, metric="cosine", 
                                                    magnificationEta=1.0, lambdaVal=0.5)
    greedyList = objFL.maximize(budget=3,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
    indices.append([i[0] for i in greedyList])
    index_list[index] = indices

In [ ]:
index_list[0]

In [ ]:
with open('logdetmi_answertss_3.pkl', 'wb') as handle:
        pickle.dump(index_list, handle)

In [ ]:
index_list={}
for i in range(0,len(finetune_data_op_embeddings), 100):
    index = int(i/100)
    #print(index)

    answer = qa[index].reshape((1, 768))
    query = qq[index].reshape((1, 768))
    
    qd = np.concatenate([answer])
    #print(len(qd))

    indices=[]
    features = finetune_data_op_embeddings[i:i+100]
    objFL = ConcaveOverModularFunction(n=100, num_queries=len(qd), data=features, 
                                                    queryData=qd, 
                                     metric="cosine", queryDiversityEta=1.0, mode=ConcaveOverModular.logarithmic)
    greedyList = objFL.maximize(budget=3,optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, verbose=False)
    indices.append([i[0] for i in greedyList])
    index_list[index] = indices

In [ ]:
index_list[0]

In [ ]:
with open('concaveovermodular_answertss_3.pkl', 'wb') as handle:
        pickle.dump(index_list, handle)